## Seismic Stack SEG-Y Import


For this notebook, you will need the segpy package to read SEGY files. Use pip to install this package.
<br>


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors, cm, pyplot as plt
% matplotlib inline
plt.rcParams['figure.figsize'] = (5, 5)
#np.set_printoptions(linewidth=120)

import colorsys as cs
from scipy.ndimage.interpolation import shift
from PIL import Image

from segpy.reader import create_reader


We now define a function to that will readin seismic data in SEGY format.

In [2]:
def read_lines(segy_file, line_type='crossline_number'):
    # read in SEGY file
    lines = []
    lines_id = []
    lines_id_unique = []
    line_length = []
    line_counter = 0
    
    with open(segy_file, 'rb') as in_file:
        segy_reader = create_reader(in_file)
        for i in segy_reader.trace_indexes():
            trace = segy_reader.trace_samples(i)
            header = segy_reader.trace_header(i)
            lines.append(list(trace))
            if line_type=='crossline_number':          #chose line_type according to the data 
                header_loc = header.crossline_number
            else:
                header_loc = header.inline_number
                
            lines_id.append(header_loc)
            
            # keep track of unique line IDs, assume that shots are sorted contiguously
            if header_loc not in lines_id_unique:
                lines_id_unique.append(header_loc)
                line_length.append(0)
                line_counter += 1
            line_length[line_counter-1] += 1

    lines_id = np.asarray(lines_id, dtype=int)
    line_length = np.asarray(line_length, dtype=int)
    lines_id_unique = np.asarray(lines_id_unique, dtype=int)
    lines = np.asarray(lines, dtype=float)

    return lines, line_length, lines_id_unique, lines_id

In [ ]:
# read in SEGY files
line_type = 'crossline_number'
path_train_img = "F:/......."
lines, line_length, lines_id_unique, lines_id = read_lines(path_train_img, line_type)

In [4]:
print("Number of traces and samples:", lines.shape)
print("Num_samples:",len(lines[1]))
print("Number of Crosslines:", line_length.shape)
print("Each lines has different number of traces: \n", line_length, "\n")
print("Crossline id:\n", lines_id_unique, "\n")
print("Line id:",lines_id.shape, "\n", lines_id)

Number of traces and samples: (72440, 480)
Num_samples: 480
Number of Crosslines: (70,)
Each lines has different number of traces: 
 [ 762  762  762  916  938  938 1061 1108 1505 1504 1460 1405 1405 1405 1404
 1500 1500 1499 1498 1451 1295 1199 1152 1105 1065 1058 1056 1068 1087 1139
 1186 1186 1186 1186 1177 1157 1148 1148 1148 1148  962  963  965  797  799
  799  801  802  803  804  805  806  807  808  808  810  810  812  812  814
  814  816  816  818  818  820  819  819  818  818] 

Crossline id:
 [11801 11901 12001 12101 12201 12301 12401 12501 12601 12701 12801 12901
 13001 13101 13201 13301 13401 13501 13601 13701 13801 13901 14001 14101
 14201 14301 14401 14501 14601 14701 14801 14901 15001 15101 15201 15301
 15401 15501 15601 15701 15801 15901 16001 16101 16201 16301 16401 16501
 16601 16701 16801 16901 17001 17101 17201 17301 17401 17501 17601 17701
 17801 17901 18001 18101 18201 18301 18401 18501 18601 18701] 

Line id: (72440,) 
 [11801 11801 11801 ..., 18701 18701 18701]


# Visualizing a seismic image

In [ ]:
plt.imshow(np.transpose(lines[0:761], (1,0)), cmap='Greys', origin='upper')

### Creating an array of seismic images
### All the 2D images are combined into a big array

In [ ]:
images = lines_to_imgs(lines, line_length, lines_id_unique, lines_id)

In [8]:
print('Final array is of shape ',images.shape)

Final array is of shape  (115, 480, 480)


### Some addition function to process or munge the images:
eg. normalizing training data, clipping the amplitude of the seismic traces, 
convert seismic image array to RGB, color saturation etc...

In [5]:
def normalize_imgs(train_data):

    mean = np.mean(train_data, axis=(1,2))  # mean for data centering
    std = np.std(train_data, axis=(1,2))  # std for data normalization
    train_data_norm = train_data
    for i in range(len(train_data)):
        train_data_norm[i] = np.subtract(train_data[i], mean[i])
        train_data_norm[i] /= std[i]
        
    return train_data_norm

In [7]:
def clip_traces_vgg(shots, clip_value):
    # clips, re-scale and re-mean traces to make it compatible with VGGNet
    # note that values are still single precision floating point numbers (float32), even though it has the same range as uint8
    clip_value = np.abs(clip_value)

    output = np.clip(shots, a_min=-clip_value, a_max=clip_value)

    # scale to twice of clip value, assuming data has mean of zero
    output = (output + clip_value) / (2.0 * clip_value)

    return 255 * output

In [16]:
def seismic_to_RGB(line, shift_value=1):
    num_imgs = np.size(line, axis=0)
    img_rows = np.size(line, axis=1)
    img_cols = np.size(line, axis=2)

    line_up = np.zeros((num_imgs, img_rows, img_cols), dtype=float)
    line_down = np.zeros((num_imgs, img_rows, img_cols), dtype=float)

    for i in range(0, num_imgs):
        for k in range(0, img_rows):
            line_up[i, k, :] = shift(line[i, k, :], shift_value, cval=0)
            line_down[i, k, :] = shift(line[i, k, :], -shift_value, cval=0)


    line_RGB = np.stack((line_down, line, line_up), axis=-1).astype(np.float32)
    print('Original shape: ', line.shape)
    print('New shape: ', line_RGB.shape)
    return line_RGB

In [19]:
def SeismicRGB_to_Saturation(line_RGB):
    num_imgs = np.size(line_RGB, axis=0)
    img_rows = np.size(line_RGB, axis=1)
    img_cols = np.size(line_RGB, axis=2)

    Temp_HSV = np.zeros((num_imgs,img_rows,img_cols,3), dtype=float)


    for i in range(num_imgs):

        im_temp = Image.fromarray(line_RGB[i].astype(np.uint8), mode='RGB')
        Temp_HSV[i] = np.array(im_temp.convert(mode='HSV'))

    Saturation = np.zeros((num_imgs,img_rows,img_cols), dtype=float)
    
    Saturation = Temp_HSV[:,:,:,2]

    return Saturation

In [ ]:
# Display a saturated line from the array

RGB_Sat = SeismicRGB_to_Saturation(Image_RGB)
plt.imshow(np.transpose(RGB_Sat[60],(1,0)).astype(np.uint8))